In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from pathlib import Path
import cv2 as cv
from feat import Detector
from feat.utils import FEAT_EMOTION_COLUMNS

In [14]:
path=Path(os.getcwd()).parent
DIR_PATH=str(path) + '\\'

In [15]:
DIR_PATH

'c:\\Users\\Audit\\Uppsala - Masters Europe\\Semester 3\\IIS\\IIS_Project\\'

In [3]:
df = Path(str(path) + '/data/extracted_df.csv')
data = pd.read_csv(df)

In [ ]:
data.columns

Index(['AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07', 'AU09', 'AU10', 'AU11',
       'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU24', 'AU25', 'AU26',
       'AU28', 'AU43', 'anger', 'disgust', 'fear', 'happiness', 'sadness',
       'surprise', 'neutral', 'input', 'valence', 'arousal', 'expression'],
      dtype='object')

In [ ]:
data['expression'].value_counts()

expression
0    410
1    336
3    166
6    159
2     89
4     72
5     53
Name: count, dtype: int64

In [4]:
df_to_work = data[['expression', 'AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07', 'AU09', 'AU10', 'AU11', 'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU24', 'AU25', 'AU26', 'AU28', 'AU43']]

In [5]:
expression = {"anger": 0, "disgust": 1, "fear": 2, "happiness": 3, "neutral": 4, "sadness": 5, "surprise": 6}

In [23]:
len(expression)

7

In [20]:
df_to_work

,expression,AU01,AU02,AU04,AU05,AU06,AU07,AU09,AU10,AU11,...,AU14,AU15,AU17,AU20,AU23,AU24,AU25,AU26,AU28,AU43
0,6,0.483270,0.240991,0.510467,0.338625,0.543061,1.0,0.564238,0.085105,0.0,...,0.402954,0.108790,0.192308,1.0,0.349684,0.021519,0.997219,0.427437,0.052202,0.366805
1,6,0.423278,0.181360,0.253868,0.270021,0.095835,0.0,0.338869,0.000630,1.0,...,0.315263,0.140137,0.470449,0.0,0.283951,0.190663,0.014161,0.097087,0.075233,0.119555
2,6,0.175819,0.165525,0.547125,0.274774,0.663774,1.0,0.603282,0.051478,1.0,...,0.313485,0.373310,0.397280,1.0,0.263181,0.417918,0.991525,0.477753,0.022979,0.561816
3,3,0.502532,0.447427,0.583796,0.438239,0.159838,0.0,0.352225,0.009576,0.0,...,0.226066,0.174352,0.358096,0.0,0.605431,0.181398,0.818789,0.161827,0.301793,0.246499
4,6,0.448395,0.388466,0.761920,0.291717,0.253567,1.0,0.414097,0.038263,0.0,...,0.255993,0.389708,0.394616,1.0,0.530567,0.260680,0.961297,0.124762,0.244503,0.370110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,3,0.300201,0.523174,0.382826,0.325378,0.124721,0.0,0.319086,0.546298,0.0,...,0.664082,0.086371,0.242000,1.0,0.542635,0.068812,0.997035,0.502084,0.039123,0.124136
1281,3,0.395263,0.168537,0.335622,0.412944,0.139587,1.0,0.461420,0.495238,1.0,...,0.200780,0.238431,0.354316,1.0,0.339715,0.265369,0.959811,0.141912,0.045829,0.137979
1282,6,0.292972,0.237607,0.403823,0.406746,0.090633,0.0,0.250610,0.117695,0.0,...,0.162623,0.071366,0.344678,1.0,0.677146,0.135040,0.791726,0.086294,0.148183,0.327276
1283,6,0.358021,0.201202,0.778042,0.362900,0.232533,0.0,0.501429,0.015568,0.0,...,0.217689,0.633931,0.378897,1.0,0.771332,0.234650,0.965866,0.260686,0.259783,0.402276


In [6]:
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch

In [7]:
class MLP(nn.Module):
    def __init__(self, features_in=2, features_out=3):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(features_in, 10),
            nn.ReLU(),
            nn.Linear(10, features_out),
        )

    def forward(self, input):
        return self.net(input)

In [21]:
class MultiEmoVA(Dataset):
    def __init__(self, data):
        super().__init__()

        # everything in pytorch needs to be a tensor
        self.inputs = torch.tensor(data.drop("expression", axis=1).to_numpy(dtype=np.float32))

        # we need to transform label (str) to a number. In sklearn, this is done internally
        #self.index2label = [label for label in data["expression"].unique()]
        #label2index = {label: i for i, label in enumerate(self.index2label)}

        self.labels = torch.tensor(data["expression"])

    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]

    def __len__(self):
        return len(self.inputs)

In [25]:
from tqdm import tqdm

dataset = MultiEmoVA(df_to_work)
K = 15
# passing a generator to random_split is similar to specifying the seed in sklearn
generator = torch.Generator().manual_seed(2023)
# we need to move our model to the correct device
cross_validation = []
acc = 0
# it is common to do a training loop multiple times, we call these 'epochs'
for k in tqdm(range(K)):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    train, test = random_split(dataset, [0.8, 0.2], generator=generator)
    
    print("Number of objects in Training set: ", len(train))
    print("Number of objects in Validation set: ", len(test))
    
    train_loader = DataLoader(train, batch_size=16, shuffle=True)
    loss_fn = nn.CrossEntropyLoss()
    model = MLP(train[0][0].shape[0], len(expression)).to(device)
    optim = torch.optim.Adam(model.parameters(), lr=0.001)

    max_epochs = 300
    for epoch in tqdm(range(max_epochs)):
        for inputs, labels in train_loader:
            # both input, output and model need to be on the same device
            inputs = inputs.to(device)
            labels = labels.to(device)

            out = model(inputs)
            loss = loss_fn(out, labels)

            loss.backward()
            optim.step()
            optim.zero_grad()
            
    # print(f"Training epoch {epoch} average loss: {loss:.4f}")
    # tell pytorch we're not training anymore
    with torch.no_grad():
        test_loader = DataLoader(test, batch_size=4)
        correct = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = model(inputs)
            # Here we go from the models output to a single class and compare to ground truth
            correct += (predictions.softmax(dim=1).argmax(dim=1) == labels).sum()
        print(f"Accuracy is: {correct / len(test) * 100:0.4f}%")
    if acc < correct / len(test) * 100:
        acc = correct / len(test) * 100
        torch.save(model, DIR_PATH + f'models/best_model_{k}.pt')
        print('Saving model! ', f'Acc of {correct / len(test) * 100} observed!')
    else:
        pass
    k_run_accuracy = correct / len(test) * 100
    cross_validation.append(k_run_accuracy)
print(f"Mean accuracy: {sum(cross_validation) / len(cross_validation):0.4f}%")

  0%|          | 0/15 [00:00<?, ?it/s]

Using device: cuda
Number of objects in Training set:  1028
Number of objects in Validation set:  257


  7%|▋         | 1/15 [00:37<08:44, 37.46s/it]

Accuracy is: 67.7043%
Saving model!  Acc of 67.70427703857422 observed!
Using device: cuda
Number of objects in Training set:  1028
Number of objects in Validation set:  257


 13%|█▎        | 2/15 [01:14<08:05, 37.33s/it]

...

Accuracy is: 70.0389%
Saving model!  Acc of 70.03890991210938 observed!
Using device: cuda
Number of objects in Training set:  1028
Number of objects in Validation set:  257


 93%|█████████▎| 14/15 [08:44<00:37, 37.50s/it]

Accuracy is: 66.9261%
Using device: cuda
Number of objects in Training set:  1028
Number of objects in Validation set:  257


100%|██████████| 15/15 [09:19<00:00, 37.31s/it]

Accuracy is: 67.3152%
Mean accuracy: 65.3437%


In [28]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 4.0916e-01,  7.3124e-01,  5.7269e-01, -5.8743e-01,  4.9214e-01,
          4.2050e-02,  9.4155e-01,  5.4522e-01,  2.2238e-01,  1.2529e+00,
          6.2073e-01, -1.7630e-02, -3.9908e-01,  4.3761e-01,  2.3788e-01,
         -7.2573e-01,  6.6051e-01,  1.0198e-01,  4.9798e-02,  5.5355e-01],
        [ 4.2388e-01,  6.3481e-01, -7.9620e-01,  1.3637e-01, -6.9853e-01,
          1.5100e-01,  3.1416e-02,  3.6951e-01,  4.4786e-02,  1.3197e+00,
          6.7393e-02, -9.3148e-02,  3.7179e-02,  2.2581e-01,  8.6299e-01,
         -8.0742e-02,  2.9910e-01, -1.4707e-01,  3.9336e-01,  2.5039e-01],
        [ 7.6911e-01, -3.8578e-02, -1.0695e+00,  4.6167e-01, -1.8829e-01,
          5.1877e-02, -9.3916e-01,  1.2354e-01,  1.6708e-01,  1.4203e+00,
          4.7551e-01, -1.0786e-01,  3.0850e-01, -1.5509e-01, -2.3416e-01,
          5.6346e-01, -3.1168e-01, -1.3029e+00,  6.8201e-01,  9.0260e-01],
        [ 8.4744e-01, -2.7222e-01,  2.1857e-01,  2.3688e-01,  6.9826e-01,
         -2.

: 

In [ ]:
#Batch size-11 , Optim - SGD, LR - 0.01

# self.net = nn.Sequential(
#     nn.Linear(features_in, 20),
#     nn.ReLU(),
#     nn.Linear(20, features_out),
# )
# 64.9806%

# self.net = nn.Sequential(
#     nn.Linear(features_in, 40),
#     nn.ReLU(),
#     nn.Linear(40, 20),
#     nn.ReLU(),
#     nn.Linear(20, 5),
#     nn.ReLU(),
#     nn.Linear(5, features_out),
# )

#63.4241%

#Batch size-16 , Optim - Adam

# self.net = nn.Sequential(
#     nn.Linear(features_in, 40),
#     nn.ReLU(),
#     nn.Linear(40, 20),
#     nn.ReLU(),
#     nn.Linear(20, 5),
#     nn.ReLU(),
#     nn.Linear(5, features_out),
# )

#63.6576%

# self.net = nn.Sequential(
#     nn.Linear(features_in, 10),
#     nn.ReLU(),
#     nn.Linear(10, features_out),
# )

#65%